# Material and Energy Balance

In [1]:
import pandas as pd
import scipy
from eosHSG import getCompound

ImportError: C extension: shares_memory not built. If you want to import pandas from the source directory, you may need to run 'python setup.py build_ext --inplace' to build the C extensions first.

In [3]:
if True:
    methane = getCompound("Methane","srk")
    T = 300 #K
    P = 1e6 #Pa
    Z = 1 
    print methane.G(T, P, Z)/1e6  #Hooray!


-44.6389660232


## Lets make as simple Pandas Dataframe

In [4]:
df_simple = pd.DataFrame({"Cities":["Gondor","Umbar","Numenor","Mordor"],
                          "Population":[20000, 10000, 100000, 500000],
                          "IsEvil":[False, True, True, True],
                          "Ruler":["Denethor","Corsair","Ar-Pharazon","Sauron"]},
                        index = ["Gnd","Umb","Num","Mor"])
df_simple

,Cities,IsEvil,Population,Ruler
Gnd,Gondor,False,20000,Denethor
Umb,Umbar,True,10000,Corsair
Num,Numenor,True,100000,Ar-Pharazon
Mor,Mordor,True,500000,Sauron


In [5]:
#A pandas dataframe is a cross between a numpy array and a python dictionary.
df_simple.loc["Gnd"]  #like a dictionary

Cities          Gondor
IsEvil           False
Population       20000
Ruler         Denethor
Name: Gnd, dtype: object

In [6]:
df_simple.loc["Gnd"]["Cities"] #like a dictionary

'Gondor'

In [7]:
df_simple["Cities"] + df_simple["Ruler"]  #like an array

Gnd        GondorDenethor
Umb          UmbarCorsair
Num    NumenorAr-Pharazon
Mor          MordorSauron
dtype: object

## Making a Process Stream

In [8]:
#Lets use 5 compounds: Nitrogen, Oxygen, Water, Carbon-Dioxide and Methane
typeeos = 'pr'
methane = getCompound("Methane", typeeos)
nitrogen = getCompound("Nitrogen", typeeos)
oxygen = getCompound("Oxygen", typeeos)
carbondioxide = getCompound("Carbon dioxide",typeeos)
water = getCompound("Water",typeeos)
dict_compounds = {"CH4":methane,
                 "N2":nitrogen,
                 "O2":oxygen,
                 "CO2":carbondioxide,
                 "H2O":water}  
#dict_compounds gives us access to all the properties of each compound

Ok.  What properties of each stream are useful to us?
<ol>
<li> Temperature K </li>
<li> Pressure Pa </li>
<li> State of the Steam (Liquid or Vapour) (we won't be bothering with mixed streams just yet) </li>
<li> Molar Flow Rates of Each Component kmol/s</li>
<li> Mol fractions of each component </li>
<li> Total Molar flow rate kmol/s </li>
<li> Total Mass Flow rate kg/s </li>
<li> Total Enthalpy in J/s </li>
<li> Total Entropy in J/Ks </li>
</ol>
<p> We will use a dataframe for this and you will see how similar this is to Excel.  Lets make the index.

In [9]:
index =  ["T", "P", "isLiq"]
index += ["MolarFlow_%s"%name for name in dict_compounds]
index += ["MolFr_%s"%name for name in dict_compounds]
index += ["TotMolFlow","TotMassFlow","Enthalpy","Entropy"]
index += ["Ncomps"]

columns = ["Air","Fuel"]
df = pd.DataFrame(index=index, columns=columns)
for col in df.columns:
    df.loc["Ncomps"][col] = len(dict_compounds)
df

,Air,Fuel
T,NaN,NaN
P,NaN,NaN
isLiq,NaN,NaN
MolarFlow_N2,NaN,NaN
MolarFlow_CO2,NaN,NaN
MolarFlow_CH4,NaN,NaN
MolarFlow_O2,NaN,NaN
MolarFlow_H2O,NaN,NaN
MolFr_N2,NaN,NaN
MolFr_CO2,NaN,NaN


So its empty.  Duh!  We have to fill it.  

In [10]:
df["Fuel"]["T"] = 300 #K
df["Fuel"]["P"] = 1.013e5 #Pa
df["Fuel"]["MolarFlow_CH4"] = 1.0 #kmol/s
df["Fuel"]["MolarFlow_N2"] = 0.0
df["Fuel"]["MolarFlow_O2"] = 0.0
df["Fuel"]["MolarFlow_CO2"] = 0.0
df["Fuel"]["MolarFlow_H2O"] = 0.0
df["Fuel"]["isLiq"] = False

df["Air"]["T"] = 300 #K
df["Air"]["P"] = 1.013e5 #Pa
df["Air"]["MolarFlow_CH4"] = 0.0 #kmol/s
df["Air"]["MolarFlow_N2"] = 3.0/0.21*0.79
df["Air"]["MolarFlow_O2"] = 3.0
df["Air"]["MolarFlow_CO2"] = 0.0
df["Air"]["MolarFlow_H2O"] = 0.0
df["Air"]["isLiq"] = False

#So what about mol fraction?  We have to write a function.
def get_molfraction(stream):
    ncomps = stream.loc["Ncomps"]
    molflow = stream.iloc[3:3+ncomps]
    totmolflow = molflow.sum()
    stream.loc["TotMolFlow"] = totmolflow 
    stream.iloc[3+ncomps:3+2*ncomps] = (stream.iloc[3:3+ncomps]/stream.iloc[3:3+ncomps].sum()).values
    return stream


In [11]:
df = df.apply(get_molfraction)
df

,Air,Fuel
T,300,300
P,101300,101300
isLiq,False,False
MolarFlow_N2,11.28571,0
MolarFlow_CO2,0,0
MolarFlow_CH4,0,1
MolarFlow_O2,3,0
MolarFlow_H2O,0,0
MolFr_N2,0.79,0
MolFr_CO2,0,0


In [12]:
names=df.index[3:8]
names

Index([u'MolarFlow_N2', u'MolarFlow_CO2', u'MolarFlow_CH4', u'MolarFlow_O2',
       u'MolarFlow_H2O'],
      dtype='object')

In [13]:
names[0]


'MolarFlow_N2'

In [14]:
names[0].split("_")

['MolarFlow', 'N2']

In [15]:
names[0].split("_")[1]

'N2'

In [16]:
Names=[names[n].split("_")[1] for n in range(5)]
Names

['N2', 'CO2', 'CH4', 'O2', 'H2O']

In [17]:
a=dict_compounds[Names[0]].Molecule.MW
a

28.013999999999999

In [18]:
MolWt=[dict_compounds[Names[n]].Molecule.MW for n in range(5)]
MolWt

[28.013999999999999,
 44.009999999999998,
 16.042999999999999,
 31.998999999999999,
 18.015000000000001]

In [19]:
ncomps = df["Air"].loc["Ncomps"]
Massflow=[df["Air"].iloc[3:3+ncomps][n]*MolWt[n] for n in range(ncomps)]
Massflow


[316.15800000000002, 0.0, 0.0, 95.997, 0.0]

In [20]:
TotMassFlow=sum(Massflow)
TotMassFlow

412.15500000000003

In [21]:
df["Air"].loc["TotMassFlow"]= TotMassFlow

In [22]:
df

,Air,Fuel
T,300,300
P,101300,101300
isLiq,False,False
MolarFlow_N2,11.28571,0
MolarFlow_CO2,0,0
MolarFlow_CH4,0,1
MolarFlow_O2,3,0
MolarFlow_H2O,0,0
MolFr_N2,0.79,0
MolFr_CO2,0,0


In [23]:
def get_TotMassFlow(stream):
    names=df.index[3:8]
    Names=[names[n].split("_")[1] for n in range(5)]
    MolWt=[dict_compounds[Names[n]].Molecule.MW for n in range(5)]
    ncomps = stream.loc["Ncomps"]
    Massflow=[stream.iloc[3:3+ncomps][n]*MolWt[n] for n in range(ncomps)]
    TotMassFlow=sum(Massflow)
    stream.loc["TotMassFlow"]= TotMassFlow
    return stream

In [24]:
df = df.apply(get_TotMassFlow)
df

,Air,Fuel
T,300,300
P,101300,101300
isLiq,False,False
MolarFlow_N2,11.28571,0
MolarFlow_CO2,0,0
MolarFlow_CH4,0,1
MolarFlow_O2,3,0
MolarFlow_H2O,0,0
MolFr_N2,0.79,0
MolFr_CO2,0,0


In [25]:
ncomps = df["Air"].loc["Ncomps"]

In [28]:
molflows=df["Air"].iloc[3:3+ncomps]
molflows

MolarFlow_N2     11.28571
MolarFlow_CO2           0
MolarFlow_CH4           0
MolarFlow_O2            3
MolarFlow_H2O           0
Name: Air, dtype: object

In [31]:
molfrac=df["Air"].iloc[3+ncomps:3+2*ncomps]
molfrac

MolFr_N2     0.79
MolFr_CO2       0
MolFr_CH4       0
MolFr_O2     0.21
MolFr_H2O       0
Name: Air, dtype: object

In [32]:
P=df["Air"].loc["P"]
P

101300.0

In [33]:
T=df["Air"].loc["T"]
T

300

In [34]:
PP=P*molfrac
PP

MolFr_N2     80027
MolFr_CO2        0
MolFr_CH4        0
MolFr_O2     21273
MolFr_H2O        0
Name: Air, dtype: object

In [35]:
Names=[names[n].split("_")[1] for n in range(5)]

In [36]:
Names

['N2', 'CO2', 'CH4', 'O2', 'H2O']

In [46]:
zz=dict_compounds[Names[0]].Z(T,PP[0])
zz
Z=[zz]

In [48]:
def get_enthalpy(stream):
    ncomps = stream.loc["Ncomps"]
    molflows = stream.iloc[3:3+ncomps]
    T = stream.loc["T"]
    P = stream.loc["P"]
    molfracs = stream.iloc[3+ncomps:3+2*ncomps]
    PP = P*molfracs
    names = [name.split("_")[1] for name in molflows.index]
    ind = 0 if stream["isLiq"] else 1
    Z = []
    for i in xrange(ncomps):
        comp = dict_compounds[names[i]]
        zz = comp.Z(T, PP[i])
        if len(zz) == 2:
            Z.append(zz[ind])
        else:
            Z.append(zz[0])
    molarH = scipy.array([dict_compounds[names[i]].H(T, PP[i], Z[i]) for i in xrange(ncomps)])
    H = (molflows*molarH).sum()
    stream.loc["Enthalpy"] = H
    return stream

In [49]:
if __name__ == "__main__":
    df = df.apply(get_enthalpy)
    df

In [50]:
df

,Air,Fuel
T,300,300
P,101300,101300
isLiq,False,False
MolarFlow_N2,11.28571,0
MolarFlow_CO2,0,0
MolarFlow_CH4,0,1
MolarFlow_O2,3,0
MolarFlow_H2O,0,0
MolFr_N2,0.79,0
MolFr_CO2,0,0
